＜取得予定データ（最低限）＞

物件名、住所、間取り、家賃、階数

1.ライブラリを読み込む

In [1]:
from bs4 import BeautifulSoup
import requests

2.RequestsでURLにアクセスしてHTMLを解析する

In [2]:
#RequsetsでURLにアクセスしてHTMLを解析する
#変数urlにSUUMOホームページのURLを格納する
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?fw2=&mt=9999999&cn=9999999&ta=13&et=9999999&sc=13103&shkr1=03&ar=030&bs=040&ct=9999999&shkr3=03&shkr2=03&srch_navi=1&mb=0&shkr4=03&cb=0.0&page={}'

#アクセスするためのURLをtarget_urlに格納する
#SUUMOのURL末尾に「&page={}」を追加し、url.formatで{}の中に数字を追加することで、forループを使えるようにします。
target_url = url.format(1)

#print()してtarget_urlを確認する
print(target_url)

https://suumo.jp/jj/chintai/ichiran/FR301FC001/?fw2=&mt=9999999&cn=9999999&ta=13&et=9999999&sc=13103&shkr1=03&ar=030&bs=040&ct=9999999&shkr3=03&shkr2=03&srch_navi=1&mb=0&shkr4=03&cb=0.0&page=1


In [3]:
#target_URLへのアクセス結果を変数rに格納
r =requests.get(target_url)

#取得結果を解析してsoupに格納
soup = BeautifulSoup(r.text)

3.Soupから情報取得する

まずは賃貸情報がどのように格納されているか確認。
Webページで「検証」を聞くと、HTMLの構造を閲覧できます。

今回は[div]タグの[cassetteitem]に格納されているのがわかりました。

In [9]:
#cassetteクラスをもったdivタグをすべて取得して、変数contentsに格納
contents = soup.find_all("div",class_="cassetteitem")

In [10]:
#変数contentsの中身を確認する
len(contents)

30

In [11]:
#変数contentにcontentsの最初の要素を格納する
content = contents[0]

4.物件情報と部屋情報が入ったブロックを取得する

In [12]:
#物件・建物情報を変数detailに格納する
detail = content.find("div",class_="cassetteitem-detail")
#各部屋の情報を変数tabeleに格納する
table = content.find("table",class_="cassetteitem_other")

5.物件情報を取得する

In [16]:
#変数tatleに、物件名を格納する
title = detail.find("div",class_="cassetteitem_content-title").text
#変数adressに住所を格納する
address = detail.find("li",class_="cassetteitem_detail-col1").text
#変数accsessにアクセス情報を格納する
accsess = detail.find("li",class_="cassetteitem_detail-col2").text
#変数ageに築年数を格納する
age = detail.find("li",class_="cassetteitem_detail-col3").text

In [17]:
title,address,accsess,age

('東京メトロ千代田線 赤坂駅 11階建 築11年',
 '東京都港区赤坂２',
 '\n東京メトロ千代田線/赤坂駅 歩4分\n東京メトロ銀座線/溜池山王駅 歩5分\n東京メトロ南北線/六本木一丁目駅 歩10分\n',
 '\n築11年\n11階建\n')

6.部屋の情報を抽出する

In [18]:
#変数tableからすべてのtrタグを取得して、変数tr_tagsに格納
tr_tags = table.find_all("tr",class_="js-cassette_link")
#tr_targsの中から最初の１つだけtr_tagに格納
tr_tag = tr_tags[0]

In [19]:
#変数floor,price,first_fee,capacityに４つ情報を格納する
#2~6番目の"td"のタグに上記の４つの情報がある
floor,price,first_fee,capacity = tr_tag.find_all("td")[2:6]

In [20]:
floor,price,first_fee,capacity

(<td>
 											9階</td>,
 <td>
 <ul>
 <li><span class="cassetteitem_price cassetteitem_price--rent"><span class="cassetteitem_other-emphasis ui-text--bold">31.2万円</span></span></li>
 <li><span class="cassetteitem_price cassetteitem_price--administration">15000円</span></li>
 </ul>
 </td>,
 <td>
 <ul>
 <li><span class="cassetteitem_price cassetteitem_price--deposit">31.2万円</span></li>
 <li><span class="cassetteitem_price cassetteitem_price--gratuity">31.2万円</span></li>
 </ul>
 </td>,
 <td>
 <ul>
 <li><span class="cassetteitem_madori">1SLDK</span></li>
 <li><span class="cassetteitem_menseki">53.75m<sup>2</sup></span></li>
 </ul>
 </td>)

In [22]:
#変数Feeとmanagement_feeに賃料と管理費を格納する
fee,management_fee = price.find_all("li")
#変数depositとgratuityに敷金と礼金を格納する
deposit,gratuity = first_fee.find_all("li")
#変数madoriとmensekiに間取りと面積を格納する
madori,menseki = capacity.find_all("li")

In [23]:
print(fee)
print(management_fee)
print()

print(deposit)
print(gratuity)
print()

print(madori)
print(menseki)
print()

<li><span class="cassetteitem_price cassetteitem_price--rent"><span class="cassetteitem_other-emphasis ui-text--bold">31.2万円</span></span></li>
<li><span class="cassetteitem_price cassetteitem_price--administration">15000円</span></li>

<li><span class="cassetteitem_price cassetteitem_price--deposit">31.2万円</span></li>
<li><span class="cassetteitem_price cassetteitem_price--gratuity">31.2万円</span></li>

<li><span class="cassetteitem_madori">1SLDK</span></li>
<li><span class="cassetteitem_menseki">53.75m<sup>2</sup></span></li>



7.取得した情報を辞書に格納する

In [27]:
d = {
    "title":title,
    "address":address,
    "accsess":accsess,
    "age":age,
    "floor":floor.text,
    "fee":fee.text,
    "management_fee":management_fee.text,
    "deposit":deposit.text,
    "gratuity":gratuity.text,
    "madori":madori.text,
    "menseki":menseki.text
}

In [28]:
d

{'title': '東京メトロ千代田線 赤坂駅 11階建 築11年',
 'address': '東京都港区赤坂２',
 'accsess': '\n東京メトロ千代田線/赤坂駅 歩4分\n東京メトロ銀座線/溜池山王駅 歩5分\n東京メトロ南北線/六本木一丁目駅 歩10分\n',
 'age': '\n築11年\n11階建\n',
 'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t9階',
 'fee': '31.2万円',
 'management_fee': '15000円',
 'deposit': '31.2万円',
 'gratuity': '31.2万円',
 'madori': '1SLDK',
 'menseki': '53.75m2'}

8.すべての物件情報の取得

In [32]:
#変数d_listに空のリストを作成する
d_list = []

#すべての物件情報（30件）取得する
contents = soup.find_all("div",class_="cassetteitem")

#各物件情報をforループで取得する
for content in contents:
    #物件情報・部屋情報を取得する
    detail = content.find("div",class_="cassetteitem-detail")
    table = content.find("table",class_="cassetteitem_other")
    #物件情報を取得する
    title = detail.find("div",class_="cassetteitem_content-title").text 
    address = detail.find("li",class_="cassetteitem_detail-col1").text
    accsess = detail.find("li",class_="cassetteitem_detail-col2").text
    age = detail.find("li",class_="cassetteitem_detail-col3").text
    #物件情報から各部屋の情報を取得する
    tr_tags = table.find_all("tr",class_="js-cassette_link")
    for tr_tag in tr_tags:
        #部屋情報の行から欲しい情報を取得する
        floor,price,first_fee,capacity = tr_tag.find_all("td")[2:6]
        #更に細かい情報を取得する
        fee,management_fee = price.find_all("li")
        deposit,gratuity = first_fee.find_all("li")
        madori,menseki = capacity.find_all("li")
        #取得した情報を辞書に格納する
        d = {
           "title":title,
           "address":address,
           "accsess":accsess,
           "age":age,
           "floor":floor.text,
           "fee":fee.text,
           "management_fee":management_fee.text,
           "deposit":deposit.text,
           "gratuity":gratuity.text,
           "madori":madori.text,
           "menseki":menseki.text
        }
        #取得した辞書をd_listに格納する
        d_list.append(d)

        

In [33]:
from pprint import  pprint
pprint(d_list[0])
print()
pprint(d_list[1])

{'accsess': '\n東京メトロ千代田線/赤坂駅 歩4分\n東京メトロ銀座線/溜池山王駅 歩5分\n東京メトロ南北線/六本木一丁目駅 歩10分\n',
 'address': '東京都港区赤坂２',
 'age': '\n築11年\n11階建\n',
 'deposit': '31.2万円',
 'fee': '31.2万円',
 'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t9階',
 'gratuity': '31.2万円',
 'madori': '1SLDK',
 'management_fee': '15000円',
 'menseki': '53.75m2',
 'title': '東京メトロ千代田線 赤坂駅 11階建 築11年'}

{'accsess': '\n東京メトロ千代田線/赤坂駅 歩4分\n東京メトロ銀座線/溜池山王駅 歩5分\n東京メトロ南北線/六本木一丁目駅 歩10分\n',
 'address': '東京都港区赤坂２',
 'age': '\n築11年\n11階建\n',
 'deposit': '32.2万円',
 'fee': '32.2万円',
 'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t6階',
 'gratuity': '-',
 'madori': '2LDK',
 'management_fee': '15000円',
 'menseki': '55.36m2',
 'title': '東京メトロ千代田線 赤坂駅 11階建 築11年'}


9.複数のページのスクレピングを実行する

In [7]:
#ライブラリを読み込む
from time import sleep
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [8]:
#複数ページから情報を取得する
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?fw2=&mt=9999999&cn=9999999&ta=13&et=9999999&sc=13103&shkr1=03&ar=030&bs=040&ct=9999999&shkr3=03&shkr2=03&srch_navi=1&mb=0&shkr4=03&cb=0.0&page={}'

#変数d_listに空のリストを作成する
d_list = []



In [9]:
#1~3をループする
for i in range(1,10):
    print("d_listの大きさ:",len(d_list))
    #変数target_urlに、アクセス先のURLを格納する
    target_url = url.format(i)
   
    #target_URLへのアクセス結果を変数rに格納
    r =requests.get(target_url)
    
    #サーバーに負荷をかけないために、１秒おきにfor文を実行する
    sleep(1)

    #取得結果を解析してsoupに格納
    soup = BeautifulSoup(r.text)

    #すべての物件情報（30件）取得する
    contents = soup.find_all("div",class_="cassetteitem")

    #各物件情報をforループで取得する
    for content in contents:
       #物件情報・部屋情報を取得する
       detail = content.find("div",class_="cassetteitem-detail")
       table = content.find("table",class_="cassetteitem_other")
       #物件情報を取得する
       title = detail.find("div",class_="cassetteitem_content-title").text 
       address = detail.find("li",class_="cassetteitem_detail-col1").text
       accsess = detail.find("li",class_="cassetteitem_detail-col2").text
       age = detail.find("li",class_="cassetteitem_detail-col3").text
       #物件情報から各部屋の情報を取得する
       tr_tags = table.find_all("tr",class_="js-cassette_link")
       for tr_tag in tr_tags:
          #部屋情報の行から欲しい情報を取得する
          floor,price,first_fee,capacity = tr_tag.find_all("td")[2:6]
          #更に細かい情報を取得する
          fee,management_fee = price.find_all("li")
          deposit,gratuity = first_fee.find_all("li")
          madori,menseki = capacity.find_all("li")
          #取得した情報を辞書に格納する
          d = {
             "title":title,
             "address":address,
             "accsess":accsess,
             "age":age,
             "floor":floor.text,
             "fee":fee.text,
             "management_fee":management_fee.text,
             "deposit":deposit.text,
             "gratuity":gratuity.text,
             "madori":madori.text,
             "menseki":menseki.text
             }
          #取得した辞書をd_listに格納する
          d_list.append(d)


d_listの大きさ: 0
d_listの大きさ: 195
d_listの大きさ: 346
d_listの大きさ: 605
d_listの大きさ: 774
d_listの大きさ: 994
d_listの大きさ: 1127
d_listの大きさ: 1295
d_listの大きさ: 1486


In [10]:
#変数d=listを使って、データフレームを作成する
df = pd.DataFrame(d_list)

In [11]:
#データフレームの先頭５行を確認する
df.head()

,title,address,accsess,age,floor,fee,management_fee,deposit,gratuity,madori,menseki
0,東京メトロ千代田線 赤坂駅 11階建 築11年,東京都港区赤坂２,\n東京メトロ千代田線/赤坂駅 歩4分\n東京メトロ銀座線/溜池山王駅 歩5分\n東京メトロ...,\n築11年\n11階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t9階,31.2万円,15000円,31.2万円,31.2万円,2LDK,53.75m2
1,東京メトロ千代田線 赤坂駅 11階建 築11年,東京都港区赤坂２,\n東京メトロ千代田線/赤坂駅 歩4分\n東京メトロ銀座線/溜池山王駅 歩5分\n東京メトロ...,\n築11年\n11階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t6階,32.2万円,15000円,32.2万円,32.2万円,2LDK,55.36m2
2,東京メトロ銀座線 外苑前駅 地下1地上3階建 築21年,東京都港区南青山４,\n東京メトロ銀座線/外苑前駅 歩9分\n東京メトロ銀座線/表参道駅 歩10分\n東京メトロ...,\n築21年\n地下1地上3階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t1階,350万円,-,1400万円,350万円,10SLDK,629.78m2
3,東京メトロ銀座線 外苑前駅 地下1地上3階建 築21年,東京都港区南青山４,\n東京メトロ銀座線/外苑前駅 歩9分\n東京メトロ銀座線/表参道駅 歩10分\n東京メトロ...,\n築21年\n地下1地上3階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t1階,350万円,-,1400万円,350万円,10LDK,629.78m2
4,ＳＵＮ ＣＡＳＴＬＥ,東京都港区南青山４,\n東京メトロ銀座線/外苑前駅 歩8分\n東京メトロ銀座線/表参道駅 歩9分\n東京メトロ千...,\n築21年\n3階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t1階,350万円,-,1400万円,350万円,ワンルーム,629.78m2


In [18]:
#to_csv()を使って、データフレームをCSV出力する
df.to_csv("suumo_scraping2.csv",index=None,encoding="utf-8-sig")